Os seguintes scripts são uma versão Python do curso de Machine Learning do professor Andrew Ng da universidade de Stanford leccionado na plataforma Coursera.

**Nota: Todos os dados e estrutura do exercício pertencem à Universidade de Stanford**

**Ressalva:** Os scripts não estão implementados de forma modular para todas as funções serem consultadas no mesmo Jupyter Notebook - ao contrário da implementação Octave.

# Exercício 1 - Gerar Features E-mail

In [5]:
# Importar o numpy para lidar com matrizes e vectores
import numpy as np
# Importar o pandas para ler ficheiros
import pandas as pd
# Importar o matplotlib como ferramenta gráfica
import matplotlib.pyplot as plt
# Importar a library de expressões regulares
import re
import string

# Importar o tokenizer do NLTK
from nltk.tokenize import word_tokenize

# Importar o stemmer
from nltk.stem.porter import *
stemmer = PorterStemmer()

# Importar o módulo de matemática
import math 

# Importar a função de optimização do scipy
from scipy import optimize, io
from scipy.ndimage import rotate

# Importar a biblioteca da classe de modelos SVM
from sklearn.svm import LinearSVC, SVC
%matplotlib inline

In [21]:
import nltk
# Download punkt if you need to
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ivobernardo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Um dos muitos problemas que podemos resolver com machine learning é a classificação de e-mails de spam.
Neste exercício vamos usar uma Suppor Vector Machine (Máquina de Vectores Suporte) para resolver o problema. 
<br> 
<br>
Vamos analisar os dados primeiro:

In [9]:
# Ler o contéudo de um e-mail para demonstração
conteudo_ficheiro = open("emailSample1.txt", "r")
conteudo_ficheiro = (conteudo_ficheiro.read())

In [11]:
print(conteudo_ficheiro)

> Anyone knows how much it costs to host a web portal ?
>
Well, it depends on how many visitors you're expecting.
This can be anywhere from less than 10 bucks a month to a couple of $100. 
You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 
if youre running something big..

To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com




Como conseguimos transformar este ficheiro em algo que a Support Vector Machine pode entender? Grande parte dos algoritmos apenas compreende dados tabulares e é necessário transformar qualquer input num formato semelhante.
<br> 
Precisamos de transformar estas palavras em números, de alguma forma - vamos começar por ler uma lista com vocabulário (conjunto de palavras onde apenas temos as palavras mais comuns de todos os e-mails que vamos analisar à frente) e pré-processar os dados e depois criar um vector de palavras com base nessa lista. Antes de criarmos o nosso vector de palavras para cada e-mail vamos realizar algumas tarefas de pré-processamento de Linguagem Natural como: 
<br>
- Manter apenas caracteres alfanuméricos.
- Mapear e-mails ou urls.


In [16]:
def obterVocabulario(
)-> dict:
    '''
    Lê um ficheiro de vocabulário e 
    mapeia-o para um dicionário onde
    cada palavra terá um número associado.
    
    Args:
        None
    Returns:
        vocab_dict(dict): Dicionário de palavras.
    '''
    vocab_dict = {}
    
    with open("vocab.txt", "r") as vocab:
        for linha in vocab:
            vocab_dict[int((linha.split('\t'))[0]),1] = linha.split('\t')[1].replace('\n','')
            
    return vocab_dict

In [22]:
def processarEmail(
    conteudo_email: str
) -> list:
    '''
    Processo o email e devolve o vector de palavras 
    com base no vocabulario.
    
    Args:
        conteudo_email(str): Conteúdo do e-mail.
    Return:
        indice_palavras(list): Lista com os índices 
        de palavras do contéudo do e-mail.
    
    '''

    listaVocabulario = obterVocabulario()
    indice_palavras = []
    # Colocar o e-mail em minúsculas
    conteudo_email = conteudo_email.lower()
    # Substituir as tags \n
    conteudo_email = conteudo_email.replace('\n',' ')
    # Reter alfanuméricos
    conteudo_email = re.sub('<[^<>]+>', ' ', conteudo_email)
    # Substituir os números 
    conteudo_email = re.sub('[0-9]+', 'number', conteudo_email)
    # Substituir os URLS
    conteudo_email = re.sub('(http|https)://[^\s]*', 'httpaddr', conteudo_email)
    # Substituir os campos de e-mail
    conteudo_email = re.sub('[^\s]+@[^\s]+', 'emailaddr', conteudo_email)
    # Substituir o símbolo de dollar
    conteudo_email = re.sub('[$]+', 'dollar', conteudo_email)
    # Criar uma lista de palavras
    conteudo_email = word_tokenize(conteudo_email)
    
    conteudo_processado = []
    
    for el in conteudo_email:
        # Remover a pontuação
        element = (el.translate(str.maketrans('', '', string.punctuation)))
        element = re.sub(r'\W+', '', element)
        if len(element)>=1:
            conteudo_processado.append(stemmer.stem(element))

    # Pesquisar no vocabulário pelo índice correspondente
    for el in conteudo_processado:
        try:
            indice_palavras.append([k for k,v in listaVocabulario.items() if v == el][0][0])
        except:
            pass
        
    return indice_palavras

In [23]:
# Gerar o vector de índices com as palavras do e-mail
indice_palavras = processarEmail(conteudo_ficheiro)

In [24]:
# Verificar o índice de palavras
indice_palavras

[86,
 916,
 794,
 1077,
 883,
 370,
 1699,
 790,
 1822,
 1831,
 883,
 431,
 1171,
 794,
 1002,
 1893,
 1364,
 592,
 1676,
 238,
 162,
 89,
 688,
 945,
 1663,
 1120,
 1062,
 1699,
 375,
 1162,
 479,
 1893,
 1510,
 799,
 1182,
 1237,
 810,
 1895,
 1440,
 1547,
 181,
 1699,
 1758,
 1896,
 688,
 1676,
 992,
 961,
 1477,
 71,
 530,
 1699,
 531]

In [25]:
def emailFeatures(
    indice_palavras: list
) ->np.array:
    '''
    Retorna a versão vectorizada do e-mail usando 
    o vector de índices de palavras.
    
    Cada e-mail está mapeado para um vector de n palavras
    da lista de vocabulário. Neste vector, o valor é 1 
    caso a palavra esteja no texto do e-mail.
    
    Args:
        indices_palavras(list): Lista de indices de palavras.
    Returns:
        features_vetorizadas(np.array): Vector de palavras.
    '''
    
    vocabulario = obterVocabulario()
    
    features_vetorizadas = np.zeros(len(vocabulario))
    for i in range(0,len(vocabulario)):
        if i in indice_palavras:
            features_vetorizadas[i] = 1
    
    return features_vetorizadas

In [26]:
features = emailFeatures(indice_palavras)

In [27]:
print('O tamanho do vector de palavras é: {}'.format(len(features)))
print('Os elementos iguais a 1 (palavras presentes no e-mail) são: {}'.format(features.sum()))

O tamanho do vector de palavras é: 1899
Os elementos iguais a 1 (palavras presentes no e-mail) são: 45.0


# Exercício 2 - Carregar Features pré-Calculadas e treinar SVM

In [28]:
# Usar o scipy para carregar a matriz de vectores de palavras por cada e-mails
ficheiro_spam = io.loadmat('spamTrain.mat')
X = np.array(ficheiro_spam['X'])
y = np.array(ficheiro_spam['y'])

Carregamos um vector de palavras por cada um dos e-mails calculados com o vocabulário utilizado acima.
<br>
Este vector é semelhante ao corrermos a função emailFeatures para diversos e-mails com texto diferente.
<br>
Esta matriz foi fornecida pelo prof. Andrew. 

**Como no primeira parte do exercício 6, vamos treinar um modelo linear SVM.**

In [29]:
def treinoSVM(
    X: np.array, 
    y: np.array, 
    C: float,
    max_iter:int
) -> SVC:
    
    '''
    Treinar um classificador SVM.
    Utilizamos a library sklearn.
    
    Args:
        X(np.array): Vector de features original.
        y(np.array): Vector de valores target (spam/não spam).
        C(float): Parâmetro de regularização.
        max_iter(int): Número de iterações.
        
    Returns:
        classificador_svm(sklearn.base.ClassifierMixin): classificador SVM treinado.
    '''
    
    classificador_svm = SVC(C=C, kernel='linear', probability=True)
    classificador_svm.fit(X,y.reshape(len(y),))     
    
    return classificador_svm

In [30]:
# Treinar um modelo com um penalty de 0.1
C = 0.1
model = treinoSVM(X, y, C, 100)

In [31]:
# Prever a probabilidade do e-mail ser spam.
p = model.predict(X)

In [32]:
print('A taxa de acerto do modelo é: {}%'.format((p.reshape(len(p),1)==y).sum()/len(y)*100))

A taxa de acerto do modelo é: 99.825%


**A taxa de acerto é bastante elevada nos dados de treino. Vamos ver a performance no conjunto de teste para perceber a capacidade de generalização.**
<br>

In [33]:
# Usar o scipy para ler o ficheiro com os vectores de palavras do conjunto de teste
ficheiro_spam_teste = io.loadmat('spamTest.mat')
X_teste = np.array(ficheiro_spam_teste['Xtest'])
y_teste = np.array(ficheiro_spam_teste['ytest'])

In [34]:
# Calcular a probabilidade de spam / não spam baseada no modelo treinado 
p_teste = model.predict(X_teste)

In [35]:
print('Taxa de acerto no conjunto de teste é {}%'.format((p_teste.reshape(len(p_teste),1)==y_teste).sum()/len(y_teste)*100))

Taxa de acerto no conjunto de teste é 98.9%


A taxa de acerto do modelo no conjunto de validação também é bastante elevada.
<br>
**Vamos ver as palavras mais importantes para a decisão de considerar um e-mail spam. Como desenvolvemos um modelo linear, é relativamente fácil retirar estas parâmetros.**
<br>

In [36]:
vocabulario = obterVocabulario()

# Obter os coeficientos do modelo para cada feature
pesos = model.coef_[0]
pesos = dict(np.hstack((np.arange(1,1900).reshape(1899,1),pesos.reshape(1899,1))))

# Ordenar os pesos do dicionário
pesos = sorted(pesos.items(), key=lambda kv: kv[1], reverse=True)

# Verificar as palavras com mais influência
top_15 = {}
for i in pesos[:15]:
    print({v for k,v in vocabulario.items() if k[0] == i[0]})

{'our'}
{'click'}
{'remov'}
{'guarante'}
{'visit'}
{'basenumb'}
{'dollar'}
{'will'}
{'price'}
{'pleas'}
{'most'}
{'nbsp'}
{'lo'}
{'ga'}
{'hour'}
